In [1]:
import pandas as pd
import dask.dataframe as dd
cohort = pd.read_csv('/project2/wparker/cohort.csv')
vitals = dd.read_csv('/project2/wparker/SIPA_data/RCLIF_vitals.csv')


In [2]:
temp = vitals.loc[(vitals['vital_name']=='temp')]

cohort_temp=temp[temp['C19_HAR_ID'].isin(cohort['C19_HAR_ID'])]

In [3]:
cohort_temp_pd = cohort_temp.compute()
cohort_temp_pd = cohort_temp_pd[['C19_HAR_ID', 'recorded_time', 'vital_name', 'vital_value']]


In [4]:
cohort_with_temp = cohort.merge(cohort_temp_pd, on='C19_HAR_ID', how='left')


In [5]:
#calculate the difference in hours from index date (e.g., start of life support) 
# to occurrence (e.g., recorded_time)
cohort_with_temp['recorded_time'] = pd.to_datetime(cohort_with_temp['recorded_time'], format='%Y-%m-%d %H:%M:%S')
cohort_with_temp['life_support_start'] = pd.to_datetime(cohort_with_temp['life_support_start'], format='%Y-%m-%d %H:%M:%S')

cohort_with_temp['hour_diff'] = (cohort_with_temp.recorded_time-cohort_with_temp.life_support_start) / pd.Timedelta(hours=1)

#filter results for only those within 46 hrs before and 6 hours after life support
cohort_with_temp = cohort_with_temp.loc[(cohort_with_temp['hour_diff'] >= -42) & (cohort_with_temp['hour_diff'] <= 6)]

##round hours to the floor
import numpy as np
cohort_with_temp['hour_diff_round'] = cohort_with_temp['hour_diff'].apply(np.round)




In [6]:
cohort_with_temp_grouped = cohort_with_temp.groupby('C19_HAR_ID', 'hour_diff_round')['temp'].agg(['max'])
cohort_with_temp_grouped

ValueError: No axis named hour_diff_round for object type DataFrame

In [ ]:
df_temp = df.groupby('C19_HAR_ID')['recorded_time'].agg(['min'])